In [ ]:
from policyengine_core.model_api import *
from policyengine_us import *
from scipy.optimize import differential_evolution

INCOME_ELASTICITY = -0.05
SUBSTITUTION_ELASTICITY = 0.25

def create_funding_reform(flat_tax_rate: float, include_lsrs: bool = True):
    return Reform.from_dict({
        "gov.contrib.ubi_center.flat_tax.abolish_federal_income_tax": {
            "year:2023:10": True
        },
        "gov.contrib.ubi_center.flat_tax.abolish_payroll_tax": {
            "year:2023:10": True
        },
        "gov.contrib.ubi_center.flat_tax.abolish_self_emp_tax": {
            "year:2023:10": True
        },
        "gov.ssa.ssi.abolish_ssi": {
            "year:2023:10": True
        },
        "gov.usda.snap.abolish_snap": {
            "year:2023:10": True
        },
        "gov.usda.wic.abolish_wic": {
            "year:2023:10": True
        },
        "gov.contrib.ubi_center.flat_tax.rate": {
            "year:2023:10": flat_tax_rate
        },
        "gov.contrib.ubi_center.flat_tax.deduct_ptc": {
            "year:2023:10": True
        },
        "gov.usda.snap.emergency_allotment.allowed": {
            "year:2023:10": False
        },
        "simulation.reported_state_income_tax": {
            "year:2023:10": True
        },
        "gov.simulation.labor_supply_responses.income_elasticity": {
            "year:2023:10": INCOME_ELASTICITY
        } if include_lsrs else {},
        "gov.simulation.labor_supply_responses.substitution_elasticity": {
            "year:2023:10": SUBSTITUTION_ELASTICITY
        } if include_lsrs else {},
    },
    "us",
    )

def create_baseline_reform():
    return Reform.from_dict({
            "gov.usda.snap.emergency_allotment.allowed": {
                "year:2023:10": False
            },
            "simulation.reported_state_income_tax": {
                "year:2023:10": True
            },
        },
        "us",
    )

baseline = Microsimulation(reform=create_baseline_reform(), dataset="enhanced_cps_2023")

class BlankSlatePolicy:
    young_child: float = 0
    older_child: float = 0
    young_adult: float = 0
    adult: float = 0
    senior: float = 0
    flat_tax_rate: float = 0.40

    def __init__(self, flat_tax_rate: float = 0.40, year: int = 2024, include_lsrs: bool = True):
        self.baseline = baseline
        self.blank_slate_funded = Microsimulation(reform=create_funding_reform(flat_tax_rate, include_lsrs), dataset="enhanced_cps_2023")
        self.baseline.default_calculation_period = year
        self.blank_slate_funded.default_calculation_period = year
        self.df = self.create_dataframe()
        self.ubi_funding = self.get_ubi_funding()
        self.flat_tax_rate = flat_tax_rate
        self.include_lsrs = include_lsrs

    def create_dataframe(self) -> pd.DataFrame:
        age = self.baseline.calc("age").values
        df = pd.DataFrame(
            dict(
                baseline_net_income=self.baseline.calc(
                    "household_net_income"
                ).values,
                baseline_tax=self.baseline.calculate(
                    "household_tax"
                ).values,
                baseline_benefits=self.baseline.calculate(
                    "household_benefits"
                ).values,
                count_young_child=self.baseline.map_result(
                    age < 6, "person", "household"
                ),
                count_older_child=self.baseline.map_result(
                    (age >= 6) & (age < 18), "person", "household"
                ),
                count_adult=self.baseline.map_result(
                    (age >= 18) & (age < 65), "person", "household"
                ),
                count_senior=self.baseline.map_result(
                    age >= 65, "person", "household"
                ),
                count_person=self.baseline.map_result(
                    age >= 0, "person", "household"
                ),
                funded_net_income=self.blank_slate_funded.calculate(
                    "household_net_income"
                ).values,
                funded_tax=self.blank_slate_funded.calculate(
                    "household_tax"
                ).values,
                funded_benefits=self.blank_slate_funded.calculate(
                    "household_benefits"
                ).values,
                count_disabled=self.baseline.calculate("is_ssi_disabled", map_to="household").values,
                weight=self.baseline.calculate("household_weight").values,
            )
        )

        df["total_employment_income"] = self.baseline.calculate("employment_income", map_to="household").values
        return df

    def get_ubi_funding(self) -> float:
        return (
            (self.df.funded_tax - self.df.funded_benefits - self.df.baseline_tax + self.df.baseline_benefits)
            * self.df.weight
        ).sum()

    def get_senior_amount(
        self,
        young_child: float,
        older_child: float,
        adult: float,
        disabled: float,
        funding_offset: float = 0,
    ) -> float:
        return (
            self.ubi_funding + funding_offset
            - young_child * (self.df.count_young_child * self.df.weight).sum()
            - older_child * (self.df.count_older_child * self.df.weight).sum()
            - adult * (self.df.count_adult * self.df.weight).sum()
            - disabled * (self.df.count_disabled * self.df.weight).sum()
        ) / (self.df.count_senior * self.df.weight).sum()

    def mean_percentage_loss(
        self,
        young_child: float,
        older_child: float,
        adult: float,
        disabled: float,
        return_extras: bool = False
    ) -> float:
        senior_amount = self.get_senior_amount(
            young_child, older_child, adult, disabled,
        )
        final_net_income = (
            self.df.funded_net_income
            + self.df.count_young_child * young_child
            + self.df.count_older_child * older_child
            + self.df.count_adult * adult
            + self.df.count_senior * senior_amount
            + self.df.count_disabled * disabled
        )
        gain = final_net_income - self.df.baseline_net_income
        income_rel_change = final_net_income / self.df.funded_net_income - 1
        income_rel_change_c = np.clip(income_rel_change, -1, 1)
        income_effect = income_rel_change_c * self.df.total_employment_income * INCOME_ELASTICITY
        employment_income_change = income_effect
        MTR_GUESS = self.flat_tax_rate
        rough_net_income_change_from_lsr = employment_income_change * (1 - MTR_GUESS)
        rough_tax_change_from_lsr = employment_income_change * MTR_GUESS
        total_ubi_funding_change = (rough_tax_change_from_lsr * self.df.weight).sum()
        if self.include_lsrs:
            senior_amount = self.get_senior_amount(
                young_child, older_child, adult, disabled, total_ubi_funding_change
            )
        final_net_income = (
            self.df.funded_net_income
            + self.df.count_young_child * young_child
            + self.df.count_older_child * older_child
            + self.df.count_adult * adult
            + self.df.count_senior * senior_amount
            + self.df.count_disabled * disabled
        )
        gain = final_net_income - self.df.baseline_net_income + rough_net_income_change_from_lsr
        if not self.include_lsrs:
            gain = gain - rough_net_income_change_from_lsr
        absolute_loss = np.maximum(0, -gain)
        pct_loss = absolute_loss / np.maximum(100, self.df.baseline_net_income)
        average = np.average(
            pct_loss, weights=self.df.weight * self.df.count_person
        )
        if return_extras:
            return average, senior_amount, total_ubi_funding_change
        return average

    def solve(self, return_amounts: bool = False, return_loss: bool = False) -> dict:
        (
            self.young_child,
            self.older_child,
            self.adult,
            self.disabled,
        ) = differential_evolution(
            lambda x: self.mean_percentage_loss(*x),
            bounds=[(0, 30e3 * self.flat_tax_rate)] * 4,
            maxiter=int(1e2),
            seed=0,
        ).x
        _, self.senior, _ = self.mean_percentage_loss(
            self.young_child, self.older_child, self.adult, self.disabled, return_extras=True
        )
        self.reform = Reform.from_dict(
            {
                **create_funding_reform(self.flat_tax_rate).parameter_values,
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount": {
                    "year:2023:10": self.young_child
                },
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount": {
                    "year:2023:10": self.older_child
                },
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount": {
                    "year:2023:10": self.adult
                },
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[3].amount": {
                    "year:2023:10": self.adult
                },
                "gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount": {
                    "year:2023:10": self.senior
                },
                "gov.contrib.ubi_center.basic_income.amount.person.disability": {
                    "year:2023:10": self.disabled
                },
            },
            "us",
        )
        if not return_amounts and not return_loss:
            return self.reform
        
        data = dict(reform=self.reform)

        if return_amounts:
            data["amounts"] = dict(
                young_child=self.young_child,
                older_child=self.older_child,
                adult=self.adult,
                senior=self.senior,
                disabled=self.disabled,
            )
        
        if return_loss:
            data["loss"] = self.mean_percentage_loss(
                self.young_child, self.older_child, self.adult, self.disabled
            )
        
        public_reform = Reform.from_dict(
            {
                path: value for path, value in self.reform.parameter_values.items()
                if path.startswith("gov")
            },
            "us",
        )
        data["id"] = public_reform.api_id
        data["reform"] = public_reform
        
        return data

In [1]:
import requests
import pandas as pd
import time

def get_basic_income_amounts(policy_id):
    # API endpoint URL
    api_url = f"https://api.policyengine.org/us/policy/{policy_id}"

    # Making a GET request to the API
    response = requests.get(api_url)

    if response.status_code == 200:
        # Parsing the JSON response
        policy_data = response.json()

        # Extracting and renaming the basic income amounts
        basic_income_amounts = {}
        age_group_mapping = {
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[0].amount": "ubi_0_5",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[1].amount": "ubi_6_17",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[2].amount": "ubi_18_64",
            "gov.contrib.ubi_center.basic_income.amount.person.by_age[4].amount": "ubi_65_plus",
            "gov.contrib.ubi_center.basic_income.amount.person.disability": "ubi_disability"
        }

        for key, value in policy_data["result"]["policy_json"].items():
            if key in age_group_mapping:
                # Assuming the amount is the first value in the dictionary
                amount = next(iter(value.values()))
                basic_income_amounts[age_group_mapping[key]] = amount
        return basic_income_amounts
    else:
        return None

# Read the CSV file into a DataFrame
df = pd.read_csv('loss_by_rate.csv')

def get_json_from_id(id):
    res = requests.get(f"https://api.policyengine.org/us/economy/{id}/over/2?time_period=2024&region=enhanced_us").json()

    while res["status"] == "computing":
        time.sleep(5)
        res = requests.get(f"https://api.policyengine.org/us/economy/{id}/over/2?time_period=2024&region=enhanced_us").json()
    
    return res["result"]


In [2]:
# Fetch and add basic income amounts for each reform ID
json_data = {}

for i, row in df.iterrows():
    reform_id = int(row['rounded_ubi_api_id'])
    print(reform_id)
    amounts = get_basic_income_amounts(reform_id)
    if amounts:
        for column_name, amount in amounts.items():
            df.loc[i, column_name] = amount
    json_data[reform_id] = get_json_from_id(reform_id)


df.to_csv('ubi_amounts.csv', index=False)
print(json_data)

42843
42844
42845
42846
42847
42848
42849
42850
42851
42852
42853
42854
42855
42856
42857
42858
42859


KeyboardInterrupt: 

In [3]:
import json

# Write the dictionary to disk as JSON
with open("json_data.json", "w") as file:
    json.dump(json_data, file)


In [4]:
import json

# Load the JSON file as a dictionary
with open("json_data.json", "r") as file:
    json_data = json.load(file)

# Print the dictionary
print(json_data)


{'42843': {'budget': {'baseline_net_income': 11960548612504.346, 'benefit_spending_impact': 1647098001884.5273, 'budgetary_impact': 11796789632.61084, 'households': 134663334.54167563, 'state_tax_revenue_impact': 0.0, 'tax_revenue_impact': 1658894791517.1382}, 'decile': {'average': {'1': 4818.422874405689, '10': 1447.5518621563926, '2': 3270.7464003411555, '3': 1934.981687166351, '4': 843.1499615263713, '5': -865.3422572166522, '6': -2929.7743648762053, '7': -5861.31794942731, '8': -7817.053189444653, '9': -10859.867292481309}, 'relative': {'1': 1.6036084700984756, '10': 0.003118725145287242, '2': 0.10154581823087647, '3': 0.04404145487125268, '4': 0.015535132542524761, '5': -0.013336952399254831, '6': -0.03798883959694355, '7': -0.06319506891073584, '8': -0.0686128927772255, '9': -0.07363681499650471}}, 'detailed_budget': {}, 'inequality': {'gini': {'baseline': 0.5275276822147373, 'reform': 0.5032447186436113}, 'top_10_pct_share': {'baseline': 0.3844596297892297, 'reform': 0.381686316

In [5]:
import plotly.express as px

# Melt the DataFrame to make it suitable for Plotly
melted_df = df.sort_values("flat_tax_rate").melt(id_vars=['flat_tax_rate'], 
                    value_vars=['ubi_0_5', 'ubi_6_17', 'ubi_18_64', 'ubi_65_plus', 'ubi_disability'],
                    var_name='UBI Category', value_name='Amount')

# Create the plot
fig = px.line(melted_df, x='flat_tax_rate', y='Amount', color='UBI Category',
              labels={'flat_tax_rate': 'Flat Tax Rate', 'Amount': 'UBI Amount'},
              title='UBI Amounts by Flat Tax Rate')

# Show the plot
fig.show()

In [6]:
import pandas as pd

def create_dataframes_from_json(json_data, df):
    dataframes = {}
    for api_id, metrics in json_data.items():
        for metric_name, metric_data in metrics.items():
            if metric_name not in dataframes:
                dataframes[metric_name] = pd.DataFrame()

            # Flatten the metric data and add API ID
            flat_data = pd.DataFrame([{**{"rounded_ubi_api_id": api_id}, **metric_data}])
            dataframes[metric_name] = pd.concat([dataframes[metric_name], flat_data], ignore_index=True)

    # Ensure data type consistency for merging
    df['rounded_ubi_api_id'] = df['rounded_ubi_api_id'].astype(str)

    # Merge each dataframe with the df dataframe to add the flat_tax_rate
    for metric_name, metric_df in dataframes.items():
        metric_df['rounded_ubi_api_id'] = metric_df['rounded_ubi_api_id'].astype(str)
        merged_df = metric_df.merge(df[['rounded_ubi_api_id', 'flat_tax_rate']], on='rounded_ubi_api_id', how='left')
        # Sort the DataFrame by flat_tax_rate
        dataframes[metric_name] = merged_df.sort_values(by='flat_tax_rate')

    return dataframes

# Assuming 'json_data' is your JSON data and 'df' is your existing DataFrame
dfs = create_dataframes_from_json(json_data, df)

# Accessing and printing a specific dataframe, for example, 'budget'
budget_df = dfs.get('budget', None)
if budget_df is not None:
    print(budget_df)


   rounded_ubi_api_id  baseline_net_income  benefit_spending_impact  \
9               42852         1.196055e+13             6.494602e+10   
10              42853         1.196055e+13             6.913572e+10   
11              42854         1.196055e+13             9.877809e+10   
12              42855         1.196055e+13             1.924121e+11   
13              42856         1.196055e+13             3.300214e+11   
14              42857         1.196055e+13             4.470682e+11   
15              42858         1.196055e+13             5.896395e+11   
7               42850         1.196055e+13             1.132128e+12   
5               42848         1.196055e+13             1.266689e+12   
3               42846         1.196055e+13             1.395907e+12   
1               42844         1.196055e+13             1.525258e+12   
0               42843         1.196055e+13             1.647098e+12   
2               42845         1.196055e+13             1.797125e+12   
4     

In [7]:
px.line(budget_df, "flat_tax_rate", "tax_revenue_impact")

In [17]:
dfs["labour_supply_response"]["total_change"]

9     4.224464e+11
10    3.915554e+11
11    3.594026e+11
12    3.243503e+11
13    2.872289e+11
14    2.513478e+11
15    2.150846e+11
7     7.274669e+10
5     3.692493e+10
3     1.485141e+09
1    -3.455510e+10
0    -6.923369e+10
2    -1.032536e+11
4    -1.361485e+11
6    -1.644624e+11
8    -2.029874e+11
Name: total_change, dtype: float64

In [27]:
import pandas as pd
import plotly.express as px

def calculate_percentage_change(dataframe, column_name):
    data = dataframe[column_name].apply(pd.Series)
    data['% change'] = ((data['reform'] - data['baseline']) / data['baseline'])
    data['flat_tax_rate'] = dataframe['flat_tax_rate']
    return data[['flat_tax_rate', '% change']].rename(columns={'% change': f'% change {column_name}'})

# Assuming dfs['inequality'] is your DataFrame
gini_change = calculate_percentage_change(dfs['inequality'], 'gini')
top_10_pct_share_change = calculate_percentage_change(dfs['inequality'], 'top_10_pct_share')
top_1_pct_share_change = calculate_percentage_change(dfs['inequality'], 'top_1_pct_share')

labour_supply_response = dfs["labour_supply_response"][["total_change", "flat_tax_rate"]].rename(columns=dict(total_change="labor_supply"))
labour_supply_response.labor_supply /= 10320e9
# Combine the data
combined_data = gini_change
combined_data = combined_data.merge(top_10_pct_share_change, on='flat_tax_rate')
combined_data = combined_data.merge(top_1_pct_share_change, on='flat_tax_rate')
combined_data = combined_data.merge(labour_supply_response, on='flat_tax_rate')

fig = px.line(combined_data, x='flat_tax_rate', y=['% change gini', '% change top_10_pct_share', '% change top_1_pct_share'],
              labels={'value': '% Change', 'variable': 'Metric'},
              title='Percentage Change in Inequality Metrics with Respect to Tax Rate')
fig.show()

In [22]:
def calculate_percentage_change_poverty(dataframe, column_name):
    data = dataframe[column_name].apply(pd.Series)
    for category in ['adult', 'all', 'child', 'senior']:
        data[f'% change {category}'] = ((data[category].apply(lambda x: x['reform']) - 
                                         data[category].apply(lambda x: x['baseline'])) / 
                                         data[category].apply(lambda x: x['baseline']))
    data['flat_tax_rate'] = dataframe['flat_tax_rate']
    return data[['flat_tax_rate', f'% change adult', f'% change all', f'% change child', f'% change senior']]

# Assuming dfs['poverty'] is your DataFrame
poverty_change = calculate_percentage_change_poverty(dfs['poverty'], 'poverty')


fig = px.line(poverty_change, x='flat_tax_rate', y=[f'% change {category}' for category in ['adult', 'all', 'child', 'senior']],
              labels={'value': '% Change', 'variable': 'Category'},
              title='Percentage Change in Poverty Metrics with Respect to Tax Rate')
fig.show()


In [23]:
deep_poverty_change = calculate_percentage_change_poverty(dfs['poverty'], 'deep_poverty')


fig = px.line(deep_poverty_change, x='flat_tax_rate', y=[f'% change {category}' for category in ['adult', 'all', 'child', 'senior']],
              labels={'value': '% Change', 'variable': 'Category'},
              title='Percentage Change in Deep Poverty Metrics with Respect to Tax Rate')
fig.show()


In [42]:
labor_supply_change = combined_data[["flat_tax_rate", "labor_supply"]]

px.line(
    combined_data,
    x="flat_tax_rate",
    y="labor_supply"
)

In [40]:
combined_change

,flat_tax_rate,labor_supply,% change gini,% change top_10_pct_share,% change top_1_pct_share,% change adult,% change all,% change child,% change senior
9,0.20,0.000144,0.046201,0.107445,0.190664,0.767656,0.880135,0.743117,1.719004
10,0.21,-0.003348,0.047333,0.106801,0.186489,0.746571,0.862340,0.735167,1.706741
11,0.22,-0.006709,0.046770,0.105128,0.185585,0.744421,0.855016,0.703780,1.702483
12,0.23,-0.010005,0.040778,0.099262,0.181065,0.658280,0.760080,0.540114,1.650914
13,0.24,-0.013193,0.033464,0.090457,0.171761,0.472748,0.596640,0.387813,1.600051
14,0.25,-0.015936,0.027224,0.082262,0.160786,0.349652,0.476972,0.268582,1.499624
15,0.26,-0.019669,0.019211,0.072062,0.147617,0.269752,0.390925,0.196406,1.358969
7,0.30,0.007049,-0.015324,0.032039,0.106146,-0.164961,-0.128229,-0.401471,0.459109
5,0.31,0.024355,-0.023421,0.021471,0.095384,-0.269584,-0.252827,-0.561823,0.267949
3,0.32,0.031429,-0.032028,0.011950,0.085820,-0.349516,-0.359304,-0.668548,0.008191


In [39]:
import pandas as pd
import plotly.express as px

# Define a function to determine increase (1) or decrease (-1)
def increase_or_decrease(value):
    return 1 if value > 0 else -1
        

# Combine the percentage change data from poverty, deep poverty, and inequality
# Assuming dfs['poverty'] and other DataFrames are already defined
combined_change = dfs['poverty'][['flat_tax_rate']].copy()
combined_change_categorical = combined_change.copy()

# Assume you have other dataframes like gini_change, top_10_pct_share_change, etc.
for dataframe in [labor_supply_change, gini_change, top_10_pct_share_change, top_1_pct_share_change, poverty_change, deep_poverty_change]:
    dataframe.sort_values('flat_tax_rate', inplace=True)
    for col in dataframe.columns:
        if col != 'flat_tax_rate':
            if col == "labor_supply":
                combined_change_categorical[col] = dataframe[col].apply(lambda x: -1 if x > 0 else 1)
            else:
                combined_change_categorical[col] = dataframe[col].apply(increase_or_decrease)
            combined_change[col] = dataframe[col]

import pandas as pd
import plotly.figure_factory as ff

# Assume combined_change is a DataFrame with the given structure
# Mapping the percentage change to colors
combined_change_mapped = combined_change_categorical.copy()

import pandas as pd
import plotly.figure_factory as ff
import numpy as np

# Rest of your code where you define combined_change and combined_change_mapped...

# Preparing the data for the heatmap
heatmap_data = combined_change_mapped.set_index('flat_tax_rate').T

# The x and y coordinates for the heatmap should be lists or arrays
x_values = combined_change_mapped['flat_tax_rate'].tolist()
y_values = heatmap_data.index.tolist()

# The z values and annotations must be 2D arrays that match the length of x and y
#heatmap_data = heatmap_data.replace({'Increase': "+", 'Decrease': "-"})
z_values = heatmap_data.values.T  # Transposing to match the dimensions
annotations = combined_change.set_index('flat_tax_rate').T.values.T  # This should be the same shape as z_values

heatmap_annotations = pd.DataFrame(annotations, columns=y_values, index=x_values)
heatmap_annotations = heatmap_annotations.apply(lambda x: x.apply(lambda y: f"{y:+.0%}"))
# Assume combined_change is a DataFrame with the given structure
# Mapping the percentage change to colors

from policyengine_core.charts import *
y_values = [
    "Labor supply",
    "Gini index",
    "Top 10% income share",
    "Top 1% income share",
    "Adult poverty rate",
    "Poverty rate",
    "Child poverty rate",
    "Senior poverty rate",
]

# Creating the heatmap using Figure Factory to properly display categorical data
fig = ff.create_annotated_heatmap(
    z=z_values.T,  # Transposing to match the dimensions
    x=x_values,
    y=y_values,
    annotation_text=heatmap_annotations.values.T,
    colorscale=[BLUE, DARK_GRAY],
    showscale=False
)

# Update the layout to remove the color axis scale (legend)
format_fig(fig).update_layout(
    title='Change in metric under optimal UBI by flat tax rate',
    xaxis=dict(title='Flat Tax Rate', side='top', tickmode='array', tickvals=x_values, ticktext=[f"{x:.0%}" for x in x_values]),
    yaxis=dict(title='Metric', tickmode='array', tickvals=list(range(len(y_values))), ticktext=y_values),
    margin=dict(l=150, t=150),  # Adjust margins to fit y-axis labels
    height=600,
    width=1200
)

# Show the figure
fig.show()


In [ ]:
combined_change.to_csv('combined_change.csv', index=False)